# Sequential Workflows

이 노트북은 Agent Framework의 Sequential Workflow 기능을 사용하여 여러 AI 에이전트를 순차적으로 연결하는 방법을 학습합니다.

## 학습 목표
- Sequential Workflow의 개념과 작동 방식 이해
- SequentialBuilder를 사용한 에이전트 체인 구성
- 순차 실행에서 컨텍스트 전달 메커니즘
- Workflow를 Agent로 래핑하는 방법
- 커스텀 Executor와 Agent의 혼합 사용
- 스트리밍 이벤트 모니터링

## Sequential Workflow란?

Sequential Workflow는 여러 AI 에이전트를 순차적으로 실행하는 오케스트레이션 패턴입니다:

### 주요 특징:
- **순차 실행**: 에이전트들이 정의된 순서대로 실행됩니다
- **컨텍스트 공유**: 각 에이전트의 출력이 다음 에이전트의 입력이 됩니다
- **대화 누적**: 전체 대화 히스토리가 체인을 따라 전달됩니다
- **Agent로 래핑**: 전체 워크플로우를 하나의 Agent처럼 사용 가능

### 사용 사례:
- **콘텐츠 생성 → 검토 → 편집** 파이프라인
- **연구 → 분석 → 요약** 워크플로우
- **번역 → 검증 → 다듬기** 과정
- **코드 생성 → 테스트 → 최적화** 단계

### 참고 자료:
- [Sequential Workflow 샘플](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/sequential_workflow_as_agent.py)
- [Agent Framework 문서](https://github.com/microsoft/agent-framework)

## 1. 환경 설정 및 필수 라이브러리 import

In [1]:
# 필수 환경 변수 확인
import os
from dotenv import load_dotenv

load_dotenv()

# 필수 환경 변수 리스트
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME",
]

# 환경 변수 확인
print("=== 환경 변수 확인 ===")
for var in required_vars:
    value = os.getenv(var)
    status = "✅" if value else "❌"
    display_value = value[:50] + "..." if value and len(value) > 50 else value
    print(f"{status} {var}: {display_value}")

print("\n환경 변수가 설정되지 않은 경우 .env 파일을 확인하세요.")

=== 환경 변수 확인 ===
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azur...
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

환경 변수가 설정되지 않은 경우 .env 파일을 확인하세요.


In [2]:
# 필수 라이브러리 import
import asyncio
from typing import cast

from agent_framework import (
    ChatMessage,
    Role,
    SequentialBuilder,
    WorkflowOutputEvent,
    WorkflowStatusEvent,
    WorkflowRunState,
    AgentRunUpdateEvent,
    ExecutorInvokedEvent,
    ExecutorCompletedEvent
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

print("✅ 모든 라이브러리가 성공적으로 import되었습니다.")

✅ 모든 라이브러리가 성공적으로 import되었습니다.


## 2. 기본 Sequential Workflow 생성

먼저 간단한 두 에이전트 체인을 만들어 Sequential Workflow의 기본 개념을 이해해보겠습니다.

In [3]:
# Azure 인증 및 클라이언트 초기화
credential = AzureCliCredential()
chat_client = AzureOpenAIChatClient(credential=credential)

print("✅ Azure 클라이언트 초기화 완료")

✅ Azure 클라이언트 초기화 완료


In [4]:
# 기본 Sequential Workflow를 위한 두 에이전트 생성
# 1. Writer Agent - 콘텐츠 작성
writer = chat_client.create_agent(
    name="writer",
    instructions="""You are a concise copywriter. 
    Provide a single, punchy marketing sentence based on the prompt.
    Be creative and engaging while keeping it brief."""
)

# 2. Reviewer Agent - 피드백 제공
reviewer = chat_client.create_agent(
    name="reviewer",
    instructions="""You are a thoughtful reviewer. 
    Give brief, constructive feedback on the previous assistant message.
    Suggest improvements or highlight strengths."""
)

print("✅ Writer 및 Reviewer 에이전트 생성 완료")
print(f"   Writer 이름: {writer.name}")
print(f"   Reviewer 이름: {reviewer.name}")

✅ Writer 및 Reviewer 에이전트 생성 완료
   Writer 이름: writer
   Reviewer 이름: reviewer


## 3. Sequential Workflow 구성 및 실행

SequentialBuilder를 사용하여 에이전트들을 순차적으로 연결합니다.

In [5]:
# Sequential Workflow 빌드
workflow = SequentialBuilder().participants([writer, reviewer]).build()

print("✅ Sequential Workflow 구성 완료")
print("   실행 순서: Writer → Reviewer")
print("   각 에이전트의 출력이 다음 에이전트의 입력 컨텍스트에 추가됩니다.")

✅ Sequential Workflow 구성 완료
   실행 순서: Writer → Reviewer
   각 에이전트의 출력이 다음 에이전트의 입력 컨텍스트에 추가됩니다.


In [6]:
# Sequential Workflow 실행 (스트리밍)
async def run_sequential_workflow(prompt: str):
    print(f"\n🚀 Sequential Workflow 실행")
    print(f"입력: {prompt}\n")
    
    outputs = []
    current_executor = None
    
    async for event in workflow.run_stream(prompt):
        # 이벤트 타입별 처리
        match event:
            case ExecutorInvokedEvent():
                executor_name = event.executor_id
                if 'writer' in executor_name or 'reviewer' in executor_name:
                    current_executor = 'Writer' if 'writer' in executor_name else 'Reviewer'
                    print(f"🔄 {current_executor} 실행 중...")
            
            case ExecutorCompletedEvent():
                if current_executor:
                    print(f"✅ {current_executor} 완료")
            
            case AgentRunUpdateEvent():
                if hasattr(event, 'data') and event.data:
                    # 실시간 응답 출력 (선택사항)
                    pass
            
            case WorkflowOutputEvent():
                outputs.append(cast(list[ChatMessage], event.data))
            
            case WorkflowStatusEvent():
                if event.state == WorkflowRunState.IDLE:
                    print("\n🏁 Workflow 완료!\n")
                    break
    
    return outputs

# 실행
prompt = "Write a tagline for a budget-friendly eBike."
results = await run_sequential_workflow(prompt)


🚀 Sequential Workflow 실행
입력: Write a tagline for a budget-friendly eBike.

🔄 Writer 실행 중...
✅ Writer 완료
🔄 Writer 실행 중...
✅ Writer 완료
🔄 Reviewer 실행 중...
✅ Reviewer 완료
🔄 Reviewer 실행 중...
✅ Reviewer 완료
✅ Reviewer 완료

🏁 Workflow 완료!



## 4. 결과 분석 및 대화 히스토리 확인

Sequential Workflow의 출력을 분석하고 전체 대화 흐름을 확인합니다.

In [7]:
# 대화 히스토리 분석
def analyze_conversation(conversation_outputs):
    if not conversation_outputs:
        print("❌ 결과가 없습니다.")
        return
    
    # 최종 대화 가져오기
    final_conversation = conversation_outputs[-1]
    
    print("=" * 80)
    print("📋 최종 대화 히스토리")
    print("=" * 80)
    
    for i, message in enumerate(final_conversation, start=1):
        role_display = message.author_name or (
            "User" if message.role == Role.USER else "Assistant"
        )
        
        print(f"\n{'-' * 60}")
        print(f"{i:02d} [{role_display}]")
        print(f"{message.text}")
    
    print(f"\n{'-' * 60}")
    print(f"\n📊 대화 통계:")
    print(f"   - 총 메시지 수: {len(final_conversation)}")
    print(f"   - 사용자 메시지: {sum(1 for msg in final_conversation if msg.role == Role.USER)}")
    print(f"   - 어시스턴트 메시지: {sum(1 for msg in final_conversation if msg.role == Role.ASSISTANT)}")
    
    # 에이전트별 기여도 분석
    agent_contributions = {}
    for msg in final_conversation:
        if msg.author_name and msg.role == Role.ASSISTANT:
            agent_contributions[msg.author_name] = agent_contributions.get(msg.author_name, 0) + 1
    
    if agent_contributions:
        print(f"\n🤖 에이전트별 기여도:")
        for agent, count in agent_contributions.items():
            print(f"   - {agent}: {count}개 메시지")

# 결과 분석
analyze_conversation(results)

📋 최종 대화 히스토리

------------------------------------------------------------
01 [User]
Write a tagline for a budget-friendly eBike.

------------------------------------------------------------
02 [writer]
Ride farther, spend smarter—your affordable eBike adventure starts here.

------------------------------------------------------------
03 [reviewer]
Strengths:  
- The tagline is clear, concise, and emphasizes both affordability ("spend smarter") and the extended riding experience ("ride farther").
- It encourages action and a sense of adventure, which could appeal to budget-conscious consumers.

Suggestions for improvement:  
- Consider highlighting environmental benefits or ease of use for broader appeal.
- You could experiment with rhyming or alliteration for added memorability, e.g., "Budget rides, boundless strides."

------------------------------------------------------------

📊 대화 통계:
   - 총 메시지 수: 3
   - 사용자 메시지: 1
   - 어시스턴트 메시지: 2

🤖 에이전트별 기여도:
   - writer: 1개 메시지
   - revie

## 5. Workflow를 Agent로 래핑하기

Sequential Workflow 전체를 하나의 Agent처럼 사용할 수 있도록 래핑하는 방법을 학습합니다.

In [8]:
# Workflow를 Agent로 래핑
workflow_agent = workflow.as_agent(name="SequentialWorkflowAgent")

print("✅ Workflow가 Agent로 래핑되었습니다.")
print(f"   Agent 이름: {workflow_agent.name}")
print(f"   Agent ID: {workflow_agent.id}")
print("\n이제 이 Workflow를 일반 Agent처럼 사용할 수 있습니다.")

✅ Workflow가 Agent로 래핑되었습니다.
   Agent 이름: SequentialWorkflowAgent
   Agent ID: WorkflowAgent_6c3fe817

이제 이 Workflow를 일반 Agent처럼 사용할 수 있습니다.


In [9]:
# Workflow Agent 사용 예제
async def test_workflow_agent():
    test_prompts = [
        "Create a slogan for eco-friendly coffee cups.",
        "Write a tagline for a productivity app.",
        "Develop a catchphrase for online education platform."
    ]
    
    print("🧪 Workflow Agent 테스트\n")
    
    for i, prompt in enumerate(test_prompts, 1):
        print(f"테스트 {i}: {prompt}")
        
        # Agent처럼 직접 호출
        response = await workflow_agent.run(prompt)
        
        if response.messages:
            # 최종 결과만 출력 (Reviewer의 피드백)
            final_message = response.messages[-1]
            print(f"📝 최종 결과: {final_message.text[:100]}...\n")
        
        print("-" * 50)

# 테스트 실행
await test_workflow_agent()

🧪 Workflow Agent 테스트

테스트 1: Create a slogan for eco-friendly coffee cups.
📝 최종 결과: Strengths:  
- The slogan is clear and directly emphasizes sustainability and care for the environme...

--------------------------------------------------
테스트 2: Write a tagline for a productivity app.
📝 최종 결과: Strengths:
- The tagline concisely communicates the core benefits of the app: increased productivity...

--------------------------------------------------
테스트 3: Develop a catchphrase for online education platform.
📝 최종 결과: Strengths:
- The catchphrase is concise, memorable, and highlights both accessibility ("learn anywhe...

--------------------------------------------------


## 6. 커스텀 Executor와 Agent 혼합 사용

Sequential Workflow에서 일반 Agent와 커스텀 Executor를 함께 사용하는 방법을 학습합니다.

In [10]:
# 커스텀 Summarizer Executor 정의
from agent_framework import Executor, WorkflowContext, handler

class ContentSummarizerExecutor(Executor):
    """대화 내용을 요약하고 핵심 포인트를 추출하는 커스텀 Executor"""
    
    @handler
    async def summarize_conversation(
        self, 
        conversation: list[ChatMessage], 
        ctx: WorkflowContext[list[ChatMessage]]
    ) -> None:
        """대화 내용을 분석하고 요약을 추가합니다."""
        
        # 대화 분석
        user_messages = [msg.text for msg in conversation if msg.role == Role.USER]
        assistant_messages = [msg.text for msg in conversation if msg.role == Role.ASSISTANT]
        
        # 요약 생성
        summary_text = f"""
📋 대화 요약:
• 사용자 요청: {len(user_messages)}개
• AI 응답: {len(assistant_messages)}개
• 마지막 요청: {user_messages[-1] if user_messages else 'N/A'}
• 최종 결과: Writer가 콘텐츠를 작성하고 Reviewer가 피드백을 제공했습니다.
""".strip()
        
        # 요약을 대화에 추가
        summary_message = ChatMessage(
            role=Role.ASSISTANT,
            text=summary_text,
            author_name="ContentSummarizer"
        )
        
        updated_conversation = list(conversation) + [summary_message]
        await ctx.send_message(updated_conversation)

print("✅ ContentSummarizerExecutor 정의 완료")

✅ ContentSummarizerExecutor 정의 완료


In [11]:
# 혼합 Workflow 구성: Agent + Agent + Custom Executor
summarizer = ContentSummarizerExecutor(id="content_summarizer")

# 3단계 워크플로우: Writer → Reviewer → Summarizer
enhanced_workflow = SequentialBuilder().participants([
    writer,      # Agent
    reviewer,    # Agent  
    summarizer   # Custom Executor
]).build()

print("✅ 향상된 Sequential Workflow 구성 완료")
print("   실행 순서: Writer (Agent) → Reviewer (Agent) → ContentSummarizer (Executor)")

✅ 향상된 Sequential Workflow 구성 완료
   실행 순서: Writer (Agent) → Reviewer (Agent) → ContentSummarizer (Executor)


In [12]:
# 향상된 워크플로우 실행
async def run_enhanced_workflow(prompt: str):
    print(f"\n🚀 향상된 Sequential Workflow 실행")
    print(f"입력: {prompt}\n")
    
    outputs = []
    step_counter = 0
    
    async for event in enhanced_workflow.run_stream(prompt):
        match event:
            case ExecutorInvokedEvent():
                step_counter += 1
                executor_name = event.executor_id
                
                if 'writer' in executor_name.lower():
                    print(f"📝 Step {step_counter}: Writer 실행 중...")
                elif 'reviewer' in executor_name.lower():
                    print(f"🔍 Step {step_counter}: Reviewer 실행 중...")
                elif 'summarizer' in executor_name.lower():
                    print(f"📊 Step {step_counter}: ContentSummarizer 실행 중...")
            
            case ExecutorCompletedEvent():
                print(f"   ✅ 완료")
            
            case WorkflowOutputEvent():
                outputs.append(cast(list[ChatMessage], event.data))
            
            case WorkflowStatusEvent():
                if event.state == WorkflowRunState.IDLE:
                    print("\n🏁 향상된 Workflow 완료!")
                    break
    
    return outputs

# 실행
enhanced_prompt = "Create a marketing message for sustainable fashion brand."
enhanced_results = await run_enhanced_workflow(enhanced_prompt)


🚀 향상된 Sequential Workflow 실행
입력: Create a marketing message for sustainable fashion brand.

   ✅ 완료
📝 Step 2: Writer 실행 중...
   ✅ 완료
📝 Step 3: Writer 실행 중...
   ✅ 완료
🔍 Step 4: Reviewer 실행 중...
   ✅ 완료
🔍 Step 5: Reviewer 실행 중...
   ✅ 완료
📊 Step 6: ContentSummarizer 실행 중...
   ✅ 완료
   ✅ 완료

🏁 향상된 Workflow 완료!


## 7. 향상된 결과 분석

커스텀 Executor가 포함된 워크플로우의 결과를 분석합니다.

In [13]:
# 향상된 결과 분석
def analyze_enhanced_conversation(conversation_outputs):
    if not conversation_outputs:
        print("❌ 결과가 없습니다.")
        return
    
    final_conversation = conversation_outputs[-1]
    
    print("=" * 80)
    print("📋 향상된 워크플로우 결과")
    print("=" * 80)
    
    # 단계별 출력 구분
    for i, message in enumerate(final_conversation, start=1):
        author = message.author_name or (
            "User" if message.role == Role.USER else "Assistant"
        )
        
        # 아이콘 매핑
        icon_map = {
            "User": "👤",
            "writer": "📝",
            "reviewer": "🔍",
            "ContentSummarizer": "📊"
        }
        icon = icon_map.get(author, "🤖")
        
        print(f"\n{'-' * 60}")
        print(f"{i:02d} {icon} [{author}]")
        print(f"{message.text}")
    
    print(f"\n{'-' * 60}")
    
    # 워크플로우 단계 분석
    step_analysis = {
        "사용자 입력": 0,
        "콘텐츠 생성": 0,
        "검토 및 피드백": 0,
        "요약 및 분석": 0
    }
    
    for msg in final_conversation:
        if msg.role == Role.USER:
            step_analysis["사용자 입력"] += 1
        elif msg.author_name == "writer":
            step_analysis["콘텐츠 생성"] += 1
        elif msg.author_name == "reviewer":
            step_analysis["검토 및 피드백"] += 1
        elif msg.author_name == "ContentSummarizer":
            step_analysis["요약 및 분석"] += 1
    
    print(f"\n🔄 워크플로우 단계별 분석:")
    for step, count in step_analysis.items():
        if count > 0:
            print(f"   • {step}: {count}개")

# 향상된 결과 분석
analyze_enhanced_conversation(enhanced_results)

📋 향상된 워크플로우 결과

------------------------------------------------------------
01 👤 [User]
Create a marketing message for sustainable fashion brand.

------------------------------------------------------------
02 📝 [writer]
Dress for tomorrow—style that’s kind to the planet.

------------------------------------------------------------
03 🔍 [reviewer]
Strengths:  
The assistant’s message is concise, impactful, and emotionally resonant. It clearly connects the concept of sustainability with forward-thinking fashion choices. The phrase “kind to the planet” effectively communicates the brand’s eco-friendly values.

Suggestions for improvement:  
The message could benefit from slightly more specificity to further distinguish the sustainable fashion brand from competitors. Consider incorporating words like “ethical materials” or “responsibly made” to reinforce the sustainable aspect. For example:  
“Dress for tomorrow—style made with ethical materials, kind to the planet.”

-----------------

## 8. 복잡한 Sequential Workflow 예제

실제 업무에서 사용할 수 있는 더 복잡한 Sequential Workflow를 구성해보겠습니다.

In [14]:
# 복잡한 콘텐츠 생산 파이프라인을 위한 에이전트들

# 1. 연구원 Agent
researcher = chat_client.create_agent(
    name="researcher",
    instructions="""You are a research specialist.
    Gather relevant facts, statistics, and background information about the topic.
    Focus on providing accurate, up-to-date information that will help create compelling content.
    Format your response as key research findings."""
)

# 2. 콘텐츠 작성자 Agent (기존 writer 재사용하되 지시사항 업데이트)
content_writer = chat_client.create_agent(
    name="content_writer",
    instructions="""You are a professional content writer.
    Using the research provided, create engaging, well-structured content.
    Make it informative yet accessible to a general audience.
    Include compelling headlines and clear key points."""
)

# 3. 편집자 Agent
editor = chat_client.create_agent(
    name="editor",
    instructions="""You are an experienced editor.
    Review the content for clarity, grammar, flow, and impact.
    Suggest specific improvements and polish the final version.
    Ensure the content meets professional publishing standards."""
)

print("✅ 콘텐츠 생산 파이프라인 에이전트 생성 완료")
print("   - Researcher: 정보 수집 및 조사")
print("   - Content Writer: 콘텐츠 작성")
print("   - Editor: 검토 및 편집")

✅ 콘텐츠 생산 파이프라인 에이전트 생성 완료
   - Researcher: 정보 수집 및 조사
   - Content Writer: 콘텐츠 작성
   - Editor: 검토 및 편집


In [15]:
# 품질 관리 Executor 정의
class QualityAssuranceExecutor(Executor):
    """최종 품질 검사 및 메타데이터 추가"""
    
    @handler
    async def quality_check(
        self,
        conversation: list[ChatMessage],
        ctx: WorkflowContext[list[ChatMessage]]
    ) -> None:
        """콘텐츠 품질 검사 및 메타데이터 추가"""
        
        # 워드 카운트 및 품질 메트릭
        total_words = sum(len(msg.text.split()) for msg in conversation if msg.role == Role.ASSISTANT)
        agent_count = len(set(msg.author_name for msg in conversation if msg.author_name))
        
        # 품질 보고서 생성
        quality_report = f"""
🎯 품질 보증 보고서
{'=' * 40}
📊 콘텐츠 메트릭:
  • 총 단어 수: {total_words:,}개
  • 참여 에이전트: {agent_count}개
  • 처리 단계: {len([msg for msg in conversation if msg.role == Role.ASSISTANT])}단계

✅ 품질 검사 통과
  • 콘텐츠 구조: 양호
  • 전문성 수준: 높음
  • 완성도: 우수

📋 승인 상태: 발행 준비 완료
        """.strip()
        
        qa_message = ChatMessage(
            role=Role.ASSISTANT,
            text=quality_report,
            author_name="QualityAssurance"
        )
        
        final_conversation = list(conversation) + [qa_message]
        await ctx.send_message(final_conversation)

print("✅ QualityAssuranceExecutor 정의 완료")

✅ QualityAssuranceExecutor 정의 완료


## 9. 전체 콘텐츠 생산 파이프라인 실행

In [16]:
# 완전한 콘텐츠 생산 파이프라인 구성
qa_executor = QualityAssuranceExecutor(id="quality_assurance")

# 4단계 파이프라인: Research → Write → Edit → QA
content_pipeline = SequentialBuilder().participants([
    researcher,      # 조사
    content_writer,  # 작성
    editor,         # 편집
    qa_executor     # 품질보증
]).build()

print("✅ 완전한 콘텐츠 생산 파이프라인 구성 완료")
print("   단계: Research → Write → Edit → Quality Assurance")

✅ 완전한 콘텐츠 생산 파이프라인 구성 완료
   단계: Research → Write → Edit → Quality Assurance


In [17]:
# 콘텐츠 생산 파이프라인 실행
async def run_content_pipeline(topic: str):
    print(f"\n📰 콘텐츠 생산 파이프라인 시작")
    print(f"주제: {topic}\n")
    
    pipeline_stages = {
        "researcher": "🔍 조사 단계",
        "content_writer": "📝 작성 단계", 
        "editor": "✏️ 편집 단계",
        "quality_assurance": "🎯 품질보증 단계"
    }
    
    outputs = []
    current_stage = None
    
    async for event in content_pipeline.run_stream(topic):
        match event:
            case ExecutorInvokedEvent():
                executor_id = event.executor_id.lower()
                
                for stage_key, stage_name in pipeline_stages.items():
                    if stage_key in executor_id:
                        current_stage = stage_name
                        print(f"⏳ {stage_name} 진행 중...")
                        break
            
            case ExecutorCompletedEvent():
                if current_stage:
                    print(f"✅ {current_stage} 완료\n")
            
            case WorkflowOutputEvent():
                outputs.append(cast(list[ChatMessage], event.data))
            
            case WorkflowStatusEvent():
                if event.state == WorkflowRunState.IDLE:
                    print("🎉 콘텐츠 생산 파이프라인 완료!")
                    break
    
    return outputs

# 실행
content_topic = "The impact of artificial intelligence on sustainable business practices"
pipeline_results = await run_content_pipeline(content_topic)


📰 콘텐츠 생산 파이프라인 시작
주제: The impact of artificial intelligence on sustainable business practices

⏳ 🔍 조사 단계 진행 중...
✅ 🔍 조사 단계 완료

⏳ 🔍 조사 단계 진행 중...
✅ 🔍 조사 단계 완료

⏳ 📝 작성 단계 진행 중...
✅ 📝 작성 단계 완료

⏳ 📝 작성 단계 진행 중...
✅ 📝 작성 단계 완료

⏳ ✏️ 편집 단계 진행 중...
✅ ✏️ 편집 단계 완료

⏳ ✏️ 편집 단계 진행 중...
✅ ✏️ 편집 단계 완료

⏳ 🎯 품질보증 단계 진행 중...
✅ 🎯 품질보증 단계 완료

✅ 🎯 품질보증 단계 완료

🎉 콘텐츠 생산 파이프라인 완료!


## 10. 최종 결과 분석 및 파이프라인 성능 평가

In [18]:
# 콘텐츠 파이프라인 결과 종합 분석
def comprehensive_pipeline_analysis(conversation_outputs):
    if not conversation_outputs:
        print("❌ 파이프라인 결과가 없습니다.")
        return
    
    final_conversation = conversation_outputs[-1]
    
    print("=" * 80)
    print("📈 콘텐츠 생산 파이프라인 종합 분석")
    print("=" * 80)
    
    # 단계별 상세 출력
    stage_icons = {
        "User": "👤",
        "researcher": "🔍",
        "content_writer": "📝",
        "editor": "✏️",
        "QualityAssurance": "🎯"
    }
    
    print("\n📋 파이프라인 실행 결과:")
    for i, message in enumerate(final_conversation, start=1):
        author = message.author_name or (
            "User" if message.role == Role.USER else "Assistant"
        )
        icon = stage_icons.get(author, "🤖")
        
        print(f"\n{'-' * 70}")
        print(f"Step {i:02d} {icon} [{author.upper()}]")
        
        # 내용 미리보기 (너무 길면 축약)
        content = message.text
        if len(content) > 300:
            content = content[:300] + "\n\n... [내용 축약됨] ..."
        
        print(content)
    
    print(f"\n{'=' * 70}")
    
    # 파이프라인 성능 메트릭
    metrics = {
        "총 메시지 수": len(final_conversation),
        "처리 단계 수": len([msg for msg in final_conversation if msg.role == Role.ASSISTANT]),
        "참여 에이전트 수": len(set(msg.author_name for msg in final_conversation if msg.author_name)),
        "총 단어 수": sum(len(msg.text.split()) for msg in final_conversation),
        "평균 응답 길이": sum(len(msg.text.split()) for msg in final_conversation if msg.role == Role.ASSISTANT) // max(1, len([msg for msg in final_conversation if msg.role == Role.ASSISTANT]))
    }
    
    print("\n📊 파이프라인 성능 메트릭:")
    for metric, value in metrics.items():
        print(f"   • {metric}: {value:,}")
    
    # 단계별 기여도 분석
    stage_contributions = {}
    for msg in final_conversation:
        if msg.author_name and msg.role == Role.ASSISTANT:
            word_count = len(msg.text.split())
            stage_contributions[msg.author_name] = stage_contributions.get(msg.author_name, 0) + word_count
    
    if stage_contributions:
        print("\n🔄 단계별 기여도 (단어 수 기준):")
        total_words = sum(stage_contributions.values())
        for stage, words in stage_contributions.items():
            percentage = (words / total_words) * 100 if total_words > 0 else 0
            print(f"   • {stage}: {words:,}개 단어 ({percentage:.1f}%)")

# 종합 분석 실행
comprehensive_pipeline_analysis(pipeline_results)

📈 콘텐츠 생산 파이프라인 종합 분석

📋 파이프라인 실행 결과:

----------------------------------------------------------------------
Step 01 👤 [USER]
The impact of artificial intelligence on sustainable business practices

----------------------------------------------------------------------
Step 02 🔍 [RESEARCHER]
**Key Research Findings: The Impact of Artificial Intelligence on Sustainable Business Practices**

**1. AI Enhances Resource Efficiency**
- **Energy Management:** AI-driven systems help businesses optimize energy usage, lowering costs and reducing carbon emissions. For instance, Google's DeepMind r

... [내용 축약됨] ...

----------------------------------------------------------------------
Step 03 📝 [CONTENT_WRITER]
**The Impact of Artificial Intelligence on Sustainable Business Practices**

Artificial Intelligence (AI) is revolutionizing how businesses approach sustainability, offering innovative solutions to environmental and social challenges. Here’s how AI is influencing sustainable business prac

## 요약 및 다음 단계

### 학습한 내용

1. ✅ **Sequential Workflow 기본 개념**
   - SequentialBuilder를 사용한 에이전트 체인 구성
   - 순차 실행에서 컨텍스트 전달 메커니즘
   - 스트리밍 이벤트 모니터링

2. ✅ **Workflow를 Agent로 래핑**
   - `workflow.as_agent()` 메서드 사용
   - 전체 워크플로우를 단일 Agent처럼 활용
   - 재사용 가능한 컴포넌트 생성

3. ✅ **Agent와 Executor 혼합 사용**
   - 커스텀 Executor 정의 및 통합
   - Agent와 Executor의 협업 패턴
   - 복합 워크플로우 구성

4. ✅ **실제 업무 적용 예제**
   - 콘텐츠 생산 파이프라인 구현
   - 다단계 품질 관리 프로세스
   - 성능 메트릭 분석

### Sequential Workflow의 장점

- **명확한 단계 구분**: 각 에이전트의 역할이 명확하게 분리
- **컨텍스트 누적**: 이전 단계의 결과가 다음 단계에 자동 전달
- **모듈화**: 각 단계를 독립적으로 개발하고 테스트 가능
- **재사용성**: 전체 워크플로우를 Agent로 래핑하여 재활용
- **확장성**: 새로운 단계를 쉽게 추가하거나 제거 가능

### 다음 단계

1. **고급 워크플로우 패턴**
   - Concurrent Workflow (병렬 실행)
   - Conditional Workflow (조건부 분기)
   - Fan-out/Fan-in 패턴

2. **성능 최적화**
   - 워크플로우 체크포인팅
   - 오류 처리 및 복구 메커니즘
   - 메모리 효율적인 대화 관리

3. **실제 배포**
   - 프로덕션 환경 배포
   - 모니터링 및 로깅 통합
   - 워크플로우 메트릭 수집

### 참고 자료

- 📚 [Agent Framework Sequential Workflow 문서](https://github.com/microsoft/agent-framework)
- 🔧 [Workflow Builder API](https://github.com/microsoft/agent-framework/tree/main/python/packages/core/agent_framework/_workflows)
- 📖 [Sequential Workflow 샘플 코드](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/sequential_workflow_as_agent.py)
- 🧪 [Workflow 테스트 예제](https://github.com/microsoft/agent-framework/tree/main/python/packages/core/tests/workflow)

### 실습 아이디어

1. **문서 처리 파이프라인**
   - PDF 추출 → 요약 → 번역 → 검토

2. **코드 리뷰 워크플로우**
   - 코드 분석 → 보안 검사 → 성능 평가 → 개선 제안

3. **고객 지원 파이프라인**
   - 문의 분류 → 정보 수집 → 해결책 제시 → 만족도 조사

4. **데이터 분석 워크플로우**
   - 데이터 전처리 → 분석 → 시각화 → 보고서 생성

---

**🎯 핵심 포인트**: Sequential Workflow는 복잡한 업무를 여러 전문화된 에이전트로 나누어 처리할 때 매우 유용하며, 각 단계의 출력이 다음 단계의 입력이 되는 파이프라인 패턴에 최적화되어 있습니다.